## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

### Data Load

In [ ]:
# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_26.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_26.csv')

DI_train = pd.read_csv('../data/DI_train_dataset_26.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_26.csv')

In [ ]:
# ID 열을 제외한 특성과 타겟 변수 분리
IVF_X = IVF_train.drop(['임신_성공_여부', 'ID'], axis=1)
IVF_y = IVF_train['임신_성공_여부']

DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

In [ ]:
print(f"IVF_X shape: {IVF_X.shape}")
print(f"IVF_test shape: {IVF_test.drop('ID', axis=1).shape}")
print(f"DI_X shape: {DI_X.shape}")
print(f"DI_test shape: {DI_test.drop('ID', axis=1).shape}")

IVF_X shape: (250052, 77)
IVF_test shape: (87891, 77)
DI_X shape: (6289, 31)
DI_test shape: (2176, 31)


### 인코딩 

In [ ]:
IVF_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이",
    "채취_해동_차이",
    "해동_혼합_차이",
    "혼합_이식_차이",
    "이식_해동_차이"
]

In [ ]:
DI_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "정자_기증자_나이"
]

In [ ]:
# 모든 범주형 변수를 문자열로 변환
IVF_X[IVF_categorical_columns] = IVF_X[IVF_categorical_columns].astype(str)
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
IVF_test[IVF_categorical_columns] = IVF_test[IVF_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
IVF_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

IVF_X[IVF_categorical_columns] = IVF_encoder.fit_transform(IVF_X[IVF_categorical_columns])
DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
IVF_test[IVF_categorical_columns] = IVF_encoder.transform(IVF_test[IVF_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

## Modeling

In [ ]:
# 데이터 분할
IVF_X_train, IVF_X_test, IVF_y_train, IVF_y_test = train_test_split(IVF_X, IVF_y, test_size=0.2, random_state=42)
DI_X_train, DI_X_test, DI_y_train, DI_y_test = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

### DI 데이터

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
        'max_depth': trial.suggest_int('max_depth', 3, 300),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2048),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'random_state': 42,
        'objective': 'binary',
        'metric': 'auc',
        'verbose': -1   
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(DI_X_train, DI_y_train)
    
    y_pred_proba = model.predict_proba(DI_X_test)[:, 1]
    
    auc = roc_auc_score(DI_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=4000)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-11 03:40:14,290] A new study created in memory with name: no-name-e19745ab-40c2-46cb-9ef4-2bdc6aa525e2
[I 2025-02-11 03:40:35,196] Trial 0 finished with value: 0.6649183648943293 and parameters: {'n_estimators': 5993, 'max_depth': 159, 'learning_rate': 0.0003178046986028226, 'num_leaves': 1015, 'min_child_samples': 15, 'subsample': 0.8777483618264876, 'reg_alpha': 0.0014995623742751823, 'reg_lambda': 0.388360659752424}. Best is trial 0 with value: 0.6649183648943293.
[I 2025-02-11 03:40:39,480] Trial 1 finished with value: 0.6219776698084594 and parameters: {'n_estimators': 4567, 'max_depth': 424, 'learning_rate': 0.13099305172574519, 'num_leaves': 1719, 'min_child_samples': 78, 'subsample': 0.6544271114509801, 'reg_alpha': 0.0012079398706293766, 'reg_lambda': 0.2537251251625407}. Best is trial 0 with value: 0.6649183648943293.
[I 2025-02-11 03:40:45,301] Trial 2 finished with value: 0.697901465597656 and parameters: {'n_estimators': 4402, 'max_depth': 202, 'learning_rate': 

Best trial:
  Value: 0.7103570427089235
  Params: 
    n_estimators: 2222
    max_depth: 463
    learning_rate: 0.00015084144814677498
    num_leaves: 1910
    min_child_samples: 51
    subsample: 0.8067522231748544
    reg_alpha: 0.006857968790936589
    reg_lambda: 0.027708258748369345


In [9]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 4500),
        'max_depth': trial.suggest_int('max_depth', 70, 700),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2048),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 90),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 1.0, log=True),
        'random_state': 42,
        'objective': 'binary',
        'metric': 'auc',
        'verbose': -1  
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(DI_X_train, DI_y_train)
    
    y_pred_proba = model.predict_proba(DI_X_test)[:, 1]
    
    auc = roc_auc_score(DI_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-11 10:56:44,978] A new study created in memory with name: no-name-6ae8af1a-77db-4995-a64e-2ee716fd5dd6
[I 2025-02-11 10:56:49,624] Trial 0 finished with value: 0.688167058675411 and parameters: {'n_estimators': 2566, 'max_depth': 120, 'learning_rate': 0.002060516988609115, 'num_leaves': 1103, 'min_child_samples': 37, 'subsample': 0.7232754924827264, 'reg_alpha': 0.9704577730638985, 'reg_lambda': 0.25430581825662274}. Best is trial 0 with value: 0.688167058675411.
[I 2025-02-11 10:56:51,146] Trial 1 finished with value: 0.6559765595939133 and parameters: {'n_estimators': 3064, 'max_depth': 616, 'learning_rate': 0.05603040886283899, 'num_leaves': 1703, 'min_child_samples': 32, 'subsample': 0.7262870497219045, 'reg_alpha': 0.8195251402926665, 'reg_lambda': 0.26305407464126407}. Best is trial 0 with value: 0.688167058675411.
[I 2025-02-11 10:56:52,703] Trial 2 finished with value: 0.6412461872142199 and parameters: {'n_estimators': 1438, 'max_depth': 294, 'learning_rate': 0.0687

Best trial:
  Value: 0.7103513199535313
  Params: 
    n_estimators: 2854
    max_depth: 325
    learning_rate: 0.00012094001652035211
    num_leaves: 173
    min_child_samples: 51
    subsample: 0.6027952014499763
    reg_alpha: 0.0055231068883412485
    reg_lambda: 0.018414520057010636


.